In [2]:
%pylab inline
import gc
import os
import fpfs
import numpy as np
import galsim
import astropy.io.fits as pyfits
hscDir=os.path.join(os.environ['homeWrk'],'S16ACatalogs','S16AStandardV2')

Populating the interactive namespace from numpy and matplotlib


In [3]:
cd wrkDir/FPFS2/sim

/lustre/work/xiangchong.li/work/FPFS2/sim


In [11]:
psfFWHM='60'
nx      =   100
ny      =   100
ngrid   =   64
rcut    =   16
beg     =   ngrid//2-rcut
end     =   beg+2*rcut
beta    =   0.75
noiVar  =   7e-3
pixScale=   0.168
g2      =   0.00

In [11]:
FWHM    =   eval(psfFWHM)/100.
psfInt  =   galsim.Moffat(beta=3.5,fwhm=FWHM,trunc=FWHM*4.)
psfInt  =   psfInt.shear(e1=0.02,e2=-0.02)
scale   =   0.168
npoints =  20

ud      =   galsim.UniformDeviate(1)
for g1,pend  in   zip([-0.02,0.02],['g1-0000','g1-2222']):
    for radius in [0.07,0.15,0.20]:
        gal_image   =   galsim.ImageF(nx*ngrid,ny*ngrid,scale=scale)
        gal_image.setOrigin(0,0)
        for ix in range(100):
            for iy in range(100):
                igal   =   ix*100+iy
                b       =   galsim.BoundsI(ix*ngrid,(ix+1)*ngrid-1,iy*ngrid,(iy+1)*ngrid-1)
                if igal%4==0:
                    gal0     =   galsim.RandomKnots(half_light_radius=radius,npoints=npoints,flux=10.,rng=ud)
                sub_gal_image = gal_image[b]
                ang    =   igal%4*np.pi/4. * galsim.radians
                gal     =   gal0.rotate(ang)
                # Shear the galaxy
                gal     =   gal.shear(g1=g1,g2=g2)
                gal     =   galsim.Convolve([psfInt,gal])
                # Draw the galaxy image
                gal.drawImage(sub_gal_image)
                del gal
                gc.collect()
        outfname=os.path.join('small%d_psf60' %int(radius*10),'image-0-%s.fits' %pend)
        print(outfname)
        pyfits.writeto(outfname,gal_image.array,overwrite=True)
        del gal_image
        gc.collect()

small0_psf60/image-0-g1-0000.fits
small1_psf60/image-0-g1-0000.fits
small2_psf60/image-0-g1-0000.fits
small0_psf60/image-0-g1-2222.fits
small1_psf60/image-0-g1-2222.fits
small2_psf60/image-0-g1-2222.fits


In [12]:
def measureShear(psfImg,galImgAll):
    fpTask=fpfs.fpfsBase.fpfsTask(psfImg)
    # Read GAL image
    imgList=[galImgAll[j*64:(j+1)*64,i*64:(i+1)*64] for i in range(100) for j in range(100)]

    # Measure FPFS moments
    a=fpTask.measure(imgList)

    # Measure FPFS ellipticity, FPFS response
    # The weighting parameter
    C=100
    # Again, for noiseless galaxies, you do not need to set rev=True
    # to revise second-order noise bias
    b=fpfs.fpfsBase.fpfsM2E(a,C)
    # Estimate shear
    return np.average(b['fpfs_e1']),np.average(b['fpfs_RE'])
radius=0.07
psfFname=   os.path.join('small%d_psf%s/' %(int(radius*10),psfFWHM),'psf-%s.fits' %psfFWHM)
psfData   =   pyfits.getdata(psfFname)
npad       =   (ngrid-psfData.shape[0])//2
psfData2 =   np.pad(psfData,(npad+1,npad),mode='constant')
assert psfData2.shape[0]==ngrid

pend='g1-0000'
gal_all=pyfits.getdata(os.path.join('small%d_psf60' %int(radius*10),'image-3-%s.fits' %pend))
em,rm=measureShear(psfData2,gal_all)
print(gm)
pend='g1-2222'
gal_all=pyfits.getdata(os.path.join('small%d_psf60' %int(radius*10),'image-3-%s.fits' %pend))
ep,rp=measureShear(psfData2,gal_all)
print((ep-em)/(rp+rm)*2./0.04-1)
print((ep+em)/(rp+rm))

-0.019992226314277474
-0.0003932470704929303
-1.1633228695936669e-07


In [10]:
gc.collect()

26